In [1]:
import numpy as np
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt
import time
from numba import njit
%matplotlib qt

In [2]:
V=100
L=100
eps=1e-5

In [3]:
def plot_3d(grid):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(0, grid.shape[1], 1)
    y = np.arange(0, grid.shape[0], 1)
    X, Y = np.meshgrid(x, y)
    ax.contour(X[::-1], Y, grid, 50)

def plot_2d(grid, vmax=V):
    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_subplot(111)
    ax.imshow(grid, origin='lower', vmax=vmax)

In [4]:
def upscale(A,factor):
    return np.repeat(np.repeat(A,factor,0),factor,1)

In [5]:
def problem_init(grid):
    ly, lx = grid.shape
    grid[0, lx//2-1:] = np.linspace(V, 0, lx//2+1)


def problem_boundary(grid):
    ly, lx = grid.shape
    grid[0, :lx//2-1] = grid[:0:-1, lx//2-1]


def normal_compute_step(grid):
    return .25 * (grid[1:-1, 2:] + grid[1:-1, :-2]
                  + grid[2:, 1:-1] + grid[:-2, 1:-1])


def compute_grid(shape, guess=None, init=problem_init, boundary=problem_boundary, next=normal_compute_step, max_iter=1e5, eps=1e-5):
    errors = np.zeros(int(max_iter))
    N = 0
    err = eps+1

    if guess is None:
        grid = np.zeros(shape)
    else:
        grid = guess
    
    if init:
        init(grid)

    while (err > eps and N < max_iter):
        next_grid = grid.copy()
        next_grid[1:-1,1:-1] = next(grid)

        boundary(next_grid)

        err = (next_grid-grid).max()
        errors[N] = err
        grid = next_grid
        N += 1

    return grid, N, np.trim_zeros(errors)


In [12]:
start = time.time()
grid, n, err = compute_grid((L,2*L))
print(time.time() - start)
print('Iterations: ', n)
print('Error: ', err[-1])

1.7823817729949951
Iterations:  28151
Error:  9.999705394392322e-06


In [7]:
start = time.time()
grid_div_4, n, err = compute_grid((L//4,2*L//4))
print(time.time() - start)
print('Iterations: ', n)
print('Error: ', err[-1])

0.028707027435302734
Iterations:  2350
Error:  9.9973400295994e-06


In [8]:
start = time.time()
grid_div_2, n, err = compute_grid((L//2,2*L//2))
print(time.time() - start)
print('Iterations: ', n)
print('Error: ', err[-1])

0.18802499771118164
Iterations:  8329
Error:  9.993925310425311e-06


In [10]:
start = time.time()
temp = grid_div_4.copy()
temp[0,:] = 0
temp = np.roll(temp, -1, axis=0)
temp[:,1:L//4+1] = temp[:, :L//4]
temp[:,L//4+1:-1] = temp[:, L//4+2:]
grid_div_2_upscaled, n, err = compute_grid((L//2,2*L//2), guess=upscale(temp, 2))
print(time.time() - start)
print('Iterations: ', n)
print('Error: ', err[-1])

0.14666390419006348
Iterations:  6943
Error:  9.994876371877126e-06


In [11]:
start = time.time()
temp = grid_div_2_upscaled.copy()
temp[0,:] = 0
temp = np.roll(temp, -1, axis=0)
temp[:,1:L//2+1] = temp[:, :L//2]
temp[:,L//2+1:-1] = temp[:, L//2+2:]
grid_upscaled, n, err = compute_grid((L,2*L), guess=upscale(temp, 2))
print(time.time() - start)
print('Iterations: ', n)
print('Error: ', err[-1])

1.1587879657745361
Iterations:  21725
Error:  9.995172788990203e-06


In [120]:
temp = grid_div_4.copy()
temp[0,:] = 0
temp = np.roll(temp, -1, axis=0)
temp[:,1:L//4+1] = temp[:, :L//4]
temp[:,L//4+1:-1] = temp[:, L//4+2:]
# plot_2d(upscale(grid_div_4, 2))
plot_2d(upscale(temp, 2))
plot_2d(grid_div_2_upscaled)
plot_2d(grid_div_2, vmax=None)

In [130]:
np.fabs(grid-grid_upscaled).max()

0.026786019964319507

In [24]:
fig, ax = plt.subplots()

x = np.arange(L)

for i in np.arange(L,2*L-1):
    y = grid[:,i]
    plt.plot(x, y)


In [12]:
def optimized_boundry(grid):
    # grid[0, L-1:] = np.linspace(V, 0, L+1)
    grid[0, :L-1] = grid[:0:-1, L-1]


def optimized_compute():
    errors = []
    N = 0
    err = eps+1

    grid = np.zeros((L, 2*L))
    grid[0,:] = 0
    grid[L-1,:] = 0
    grid[:,0] = 0
    grid[:,2*L-1] = 0
    grid[0, L-1:] = np.linspace(V, 0, L+1)

    while (err > eps):
        N += 1
        grid_copy = grid.copy()
        grid[1:-1,1:-1] = .25 * (grid_copy[1:-1,2:] + grid_copy[1:-1, :-2] \
                              + grid_copy[2:, 1:-1] + grid_copy[:-2, 1:-1])
        grid[0, :L-1] = grid[:0:-1, L-1]

        err = (grid-grid_copy).max()
        # errors.append(err)
        last_grid = grid

    print(N)
    return grid, N, errors


def optimized_compute_step(grid):
    new_grid = np.zeros((L, 2*L))
    new_grid[1:-1, 1:-1] = 0.25 * \
        (grid[1:-1, 2:]+grid[1:-1, :-2]+grid[:-2, 1:-1]+grid[2:, 1:-1])
    return new_grid


start = time.time()
final, n, err = optimized_compute()
print(n)
print(err)
print(time.time() - start)

# plt.plot(np.arange(n), np.log(err))
# plt.plot(np.arange(L), grid[0,:L])
# plt.imshow(grid)


28151
28151
[]
2.8157589435577393


In [50]:
fig, ax = plt.subplots()

x = np.arange(L)

for i in np.arange(L,2*L-1):
    y = grid[:,i-L]
    plt.plot(x, y)


In [51]:
def optimized_boundry(grid):
    # grid[0, L-1:] = np.linspace(V, 0, L+1)
    grid[0, :L-1] = grid[:0:-1, L-1]


def optimized_compute():
    errors = []
    N = 0
    err = eps+1

    grid = np.zeros((2*L, 2*L))
    grid[L, L-1] = V
    # grid[0, L-1:] = np.linspace(V, 0, L+1)

    while (err > eps):
        N += 1
        grid_copy = grid.copy()
        grid[1:-1,1:-1] = .25 * (grid_copy[1:-1,2:] + grid_copy[1:-1, :-2] \
                              + grid_copy[2:, 1:-1] + grid_copy[:-2, 1:-1])
        grid[L, L-1] = V

        err = (grid-grid_copy).max()
        # errors.append(err)
        last_grid = grid

    print(N)
    return grid, N, errors


def optimized_compute_step(grid):
    new_grid = np.zeros((L, 2*L))
    new_grid[1:-1, 1:-1] = 0.25 * \
        (grid[1:-1, 2:]+grid[1:-1, :-2]+grid[:-2, 1:-1]+grid[2:, 1:-1])
    return new_grid


start = time.time()
grid, n, err = optimized_compute()
print(n)
print(err)
print(time.time() - start)

# plt.plot(np.arange(n), np.log(err))
# plt.plot(np.arange(L), grid[0,:L])
# plt.imshow(grid)

40301
40301
[]
10.178325891494751


In [41]:
plt.imshow(grid)

In [44]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(2*L)
Y = np.arange(L)
X, Y = np.meshgrid(X, Y)
# Z = grid + np.rot90(grid.T,2)
# Z = grid1+grid2
Z = grid[L:,:]
ax.contour(X, Y, Z, 50)

In [45]:
grid[L:,:][0, L-1:L+1]

array([100.        ,  75.02766628])

In [46]:
np.linspace(V, 0, L+1)[:5]

array([100.,  99.,  98.,  97.,  96.])

In [ ]:
for i in np.arange(1,)

In [30]:
a = np.concatenate([[grid[L:,:][0, L-1-i:2*L-1-i]] for i in range(L)])
b = np.linspace(V, 0, L+1)[:-1]
x = np.linalg.solve(a, b)

In [31]:
a = np.concatenate([[grid[L:,:][0, L-1-i:2*L-i]] for i in range(L)])
temp = np.zeros((L+1))
temp[1:] += grid[L:,:][0,:L]
a = np.concatenate((a, [temp]))
b = np.linspace(V, 0, L+1)
x = np.linalg.solve(a, b)

In [27]:
grid[L, L-1-0:2*L-0]

IndexError: index 100 is out of bounds for axis 0 with size 100

In [32]:
guess = np.zeros((L, 2*L))

for i in range(L+1):
    shifted = np.zeros((L, 2*L))
    shifted[:, i:] = x[i]*grid[L:, :2*L-i]
    guess += shifted



In [33]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(2*L)
Y = np.arange(L)[::-1]
X, Y = np.meshgrid(X, Y)
# Z = grid + np.rot90(grid.T,2)
# Z = grid1+grid2
Z = guess
ax.contour(X, Y, Z, 50)

In [34]:
guess[0,L-1:]

array([99.95017897, 98.96446061, 97.96745484, 96.97044183, 95.97342226,
       94.97639689, 93.97936643, 92.98233164, 91.98529321, 90.98825192,
       89.99120847, 88.99416364, 87.99711812, 87.00007272, 86.00302811,
       85.00598512, 84.00894442, 83.01190684, 82.01487308, 81.01784395,
       80.02082016, 79.02380254, 78.02679179, 77.02978876, 76.03279416,
       75.03580884, 74.03883352, 73.04186907, 72.0449162 , 71.04797579,
       70.05104858, 69.05413547, 68.05723718, 67.06035463, 66.06348857,
       65.06663991, 64.06980943, 63.07299806, 62.07620657, 61.07943593,
       60.08268692, 59.08596052, 58.08925753, 57.09257894, 56.09592559,
       55.09929849, 54.10269847, 53.10612659, 52.1095837 , 51.11307087,
       50.11658898, 49.12013912, 48.12372221, 47.12733936, 46.13099151,
       45.13467981, 44.13840522, 43.14216893, 42.14597193, 41.14981545,
       40.1537005 , 39.15762835, 38.16160006, 37.16561694, 36.16968007,
       35.17379081, 34.1779503 , 33.18215992, 32.18642088, 31.19

In [36]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(2*L)
Y = np.arange(L)[::-1]
X, Y = np.meshgrid(X, Y)
# Z = grid + np.rot90(grid.T,2)
# Z = grid1+grid2
Z = guess-final
ax.contour(X, Y, Z, 50)

In [44]:
np.fabs(final_guess-final).max()

0.013572860410739906

In [16]:
plot_2d(final)

In [62]:
def reg2(A,factor):
    return np.repeat(np.repeat(A,factor,0),factor,1)

In [66]:
a = np.arange(6)
a.resize((2,3))
reg2(a, 2)

array([[0, 0, 1, 1, 2, 2],
       [0, 0, 1, 1, 2, 2],
       [3, 3, 4, 4, 5, 5],
       [3, 3, 4, 4, 5, 5]])

In [67]:
a

array([[0, 1, 2],
       [3, 4, 5]])